# Certification Histories

This notebook contains analyses of visa-certification data from the [Department of Labor's Office of Foreign Labor Certification](http://www.foreignlaborcert.doleta.gov/performancedata.cfm). The code below loads the data, and then produces employer-specific and agent-specific certification counts.

### Load the data

In [1]:
import pandas as pd
import re
from IPython.display import display, HTML
pd.set_option("display.max_rows", 25)

In [2]:
decisions = pd.read_csv("../data/H-2-certification-decisions.csv", low_memory=False)\
    .rename(columns=dict(fy="fiscal_year"))

In [3]:
to_calendar_year = lambda x: pd.to_datetime(x).year if pd.notnull(x) else None
decisions["calendar_year"] = decisions["last_event_date"].apply(to_calendar_year)

### Set up aggregation/summarization functions

In [4]:
main_cols = [
    "case_no", "visa_type", "last_event_date",
    "employer_name", "employer_city", "employer_state",
    "job_title", "case_status", "n_certified"
]

In [5]:
grouping_cols = {
    "employer": [
        "employer_name",
        "employer_state",
        "worksite_state",
    ],
    "agent": [ 
        "agent_name" 
    ] 
}

In [6]:
def agg_grouped_certs(grp):
    by_year = pd.DataFrame({
        "n_certified_applications": grp.size(),
        "n_certified_visas": grp["n_certified"].sum()
    })
    return by_year

In [7]:
def summarize_certs(subset, by="employer", year_type="calendar_year"):
    certs = subset[
        (subset["n_certified"] > 0)
    ].copy()
    certs["worksite_state"] = certs["worksite_state"].fillna("[missing]")
    certs["worksite_city"] = certs["worksite_city"].fillna("[missing]")
    
    grp_cols = grouping_cols[by]
    agg = agg_grouped_certs(certs.groupby(grp_cols))\
        .reset_index()\
        .set_index(grp_cols[0])\
        .sort_values("n_certified_visas", ascending=False)
    
    by_year = agg_grouped_certs(certs.groupby(year_type))
    by_year.index = list(map(int, by_year.index))
    by_year.loc["Total"] = by_year.sum()

    display(HTML("<h4>By {0} name:</h4>".format(by)))
    display(agg.fillna(""))
    display(HTML("<h4>By {0}:</h4>".format(year_type.replace("_", " "))))
    display(by_year)

In [8]:
def search(data, column, pattern):
    return data[
        data[column].str.contains(pattern, case=False, na=False)
    ]

## Number of employers

The OFLC data does not come with unique identifiers — e.g., federal tax IDs. The code below approximates the number of unique employers certified, using compressed and shortened versions of their listed names.

In [9]:
non_alpha_pat = re.compile(r"[^a-z]+")
def normalize_name(name):
    lowercase = name.strip().lower().replace(" and ", " & ")
    compressed = re.sub(non_alpha_pat, "", lowercase)
    limit = 10
    return compressed[:limit] if len(compressed) > limit else compressed

In [10]:
decisions["employer_name_norm"] = decisions["employer_name"].fillna("").apply(normalize_name)

In [11]:
grp = decisions[
    (decisions["n_certified"] > 0) &
    (decisions["fiscal_year"] >= 2010)
].groupby("fiscal_year")

pd.DataFrame({
    "n_employer_names": grp["employer_name_norm"].nunique()
})

,n_employer_names
fiscal_year,
2010,7974
2011,7945
2012,8172
2013,8442
2014,9403
2015,10025


## Total certifications

Note: Certifications are not equivalent to visa issuances or admissions into the United States. Certifications are permissions from the Department of Labor to seek guest workers.

In [12]:
pd.DataFrame({
    "n_certified": decisions[
        (decisions["is_duplicate"] != True) &
        (decisions["fiscal_year"] >= 2010)
    ].groupby("fiscal_year")["n_certified"].sum()
})

,n_certified
fiscal_year,
2010,165799
2011,160768
2012,160695
2013,181120
2014,210377
2015,241597


## Linda White

Certifications between Jan. 1, 2010 through Dec. 31, 2012:

In [13]:
linda_white_decisions = search(decisions, "agent_name", r"Linda White|Linda D.? White")
summarize_certs(linda_white_decisions[
    (linda_white_decisions["last_event_date"] >= "2010-01-01") &
    (linda_white_decisions["last_event_date"] <= "2012-12-31") &
    (linda_white_decisions["is_duplicate"] == False)
], by="agent")

,n_certified_applications,n_certified_visas
agent_name,,
LINDA D. WHITE,368,3969
"LINDA D. WHITE / LINDA WHITE & ASSOCIATES, INC.",57,656
"LINDA D. WHITE/LINDA WHITE & ASSOCIATES, INC.",29,481
"LINDA D. WHITE / LINDA WHITE & ASSOCIATES, INC",35,464
LINDA D WHITE,26,421
LINDA WHITE,6,245
"LINDA WHITE & ASSOCIATES, INC.",22,240
"LINDA D. WHITE/ LINDA WHITE & ASSOCIATES, INC.",26,227
"LINDA D. WHITE/ LINDA WHITE & ASSOCIATES, INC",22,173


,n_certified_applications,n_certified_visas
2010,219,2745
2011,234,2550
2012,258,2778
Total,711,8073


## Fresh Harvest, 2014

In [14]:
fresh_harvest_2014 = search(decisions[decisions["calendar_year"] == 2014], "employer_name", "Fresh Harvest")
fresh_harvest_2014_ca_cert = fresh_harvest_2014[
    (fresh_harvest_2014["worksite_state"] == "CA")
]["n_certified"].sum()
summarize_certs(fresh_harvest_2014)

,employer_state,worksite_state,n_certified_applications,n_certified_visas
employer_name,,,,
"FRESH HARVEST, INC.",CA,CA,14,1253
"FRESH HARVEST, INC.",AZ,AZ,1,269


,n_certified_applications,n_certified_visas
2014,15,1522
Total,15,1522


## California H-2A certifications, 2014

In [15]:
ca_h2a_2014_cert = decisions[
    (decisions["worksite_state"] == "CA") &
    (decisions["visa_type"] == "H-2A") &
    (decisions["calendar_year"] == 2014) &
    (decisions["is_duplicate"] == False)
]["n_certified"].sum()

In [16]:
print("In 2014, the OFLC certified employers to bring {0:,d} "
      "H-2A guest workers to California, of which Fresh Harvest " 
      "accounted for {1:.0f}%.".format(ca_h2a_2014_cert, (100.0 * fresh_harvest_2014_ca_cert) / ca_h2a_2014_cert))

In 2014, the OFLC certified employers to bring 6,001 H-2A guest workers to California, of which Fresh Harvest accounted for 21%.


---

---

---